In [74]:
using NPZ
using LinearAlgebra
using Statistics

In [59]:
xinputs = npzread("../xinputs.npz")["arr_0"]
yinputs = npzread("../yinputs.npz")["arr_0"]
hyper = npzread("../hypers.npz")["arr_0"]
alphas = npzread("../alphas.npz")["arr_0"];

In [60]:
param = [0.12, 0.022, 2.7, 1.0, 0.75];

In [429]:
function x_transformation(point, x_arr)
    """Pre-whiten the input parameters.

    Args:
        point (torch.tensor): the input parameters.

    Returns:
        torch.tensor: the pre-whitened parameters.
    """

    N, M = size(x_arr)
    cov_train = cov(x_arr)
    chol_train = cholesky(cov_train).U'
    mean_train = mean(x_arr, dims=1)
    # calculate the transformed training points
    transformed = inv(chol_train) * (point .- mean_train)'

    return transformed
end

function y_transformation(point, y_arr)
    """Transform the outputs.

    Args:
        yvalues (np.ndarray): the values to be transformed

    Returns:
        np.ndarray: the transformed outputs
    """
    ylog = log.(y_arr)
    ymean = mean(ylog, dims=1)
    ystd = std(ylog, dims=1)
    return ((point .- ymean) ./ ystd)'
end

function inv_y_transformation(point, y_arr)
    """Transform the outputs.

    Args:
        yvalues (np.ndarray): the values to be transformed

    Returns:
        np.ndarray: the transformed outputs
    """
    ylog = log.(y_arr)
    ystd = std(ylog, dims=1)
    ymean = mean(ylog, dims=1)
    return exp.(ystd .* preds' .+ ymean)
end

inv_y_transformation (generic function with 2 methods)

In [430]:
xtrain = x_transformation(xinputs, xinputs)
ytrain = y_transformation(yinputs, yinputs);

In [431]:
function get_kernel(arr1, arr2, hyper)
    """Compute the kernel matrix between the sets of training cosmoligical parameters
       and the desired set of cosmological parameters.
    Args:
        x1 (np.ndarray): [N x d] tensor of points.
        x2 (np.ndarray): [M x d] tensor of points.
        hyper (np.ndarray): [d+1] tensor of hyperparameters.
    Returns:
        np.ndarray: a tensor of size [N x M] containing the kernel matrix.
    """
    
    arr1_w = @.(arr1/exp(hyper[2:6]))
    arr2_w = @.(arr2/exp(hyper[2:6]))
    
    # compute the pairwise distance
    term1 = sum(arr1_w.^2, dims=1)
    term2 = 2 * arr1_w' * arr2_w
    term3 = sum(arr2_w.^2, dims=1)'
    dist = term1 - term2' .+ term3

    # compute the kernel
    kernel = @.(exp(hyper[1]) * exp(-0.5 * dist))

    return kernel
end

get_kernel (generic function with 1 method)

In [432]:
preds = []
for i in 1:40
    kernel = get_kernel(xtrain, param, hyper[i, :])
    mean = dot(vec(kernel), vec(alphas[i,:]))
    push!(preds, mean)
end

In [435]:
Pk = vec(inv_y_transformation(preds, yinputs))

40-element Vector{Float64}:
   3422.6928962698585
   4741.341069854786
   6634.357757713101
   9269.603527774409
  12956.205962219563
  18109.700422444228
  25012.099818223498
  34804.37073537634
  48974.04493493459
  67959.0526679225
  93641.14503262802
 130301.78251759238
 175338.60489804772
      ⋮
  88953.1563376307
  58205.12326540685
  34244.48164934919
  20034.96106569752
  11576.253737395122
   6399.914252942181
   3502.4068895960154
   2000.2365850867893
    998.7537880946642
    525.0049265753239
    273.67151022613155
    139.45020874823427